In [ ]:
import asyncio

import zmq
# publisher = zmq.socket('pub')

from kucoin_futures.client import WsToken
from kucoin_futures.ws_client import KucoinFuturesWsClient

from dotmap import DotMap

from threading import Thread

In [ ]:
zContext = zmq.asyncio.Context()
socket = zContext.socket(zmq.PUB)
socket.bind("tcp://*:5556")

In [ ]:
symbols = ['XBTUSDTM', "ALGOUSDtM", 'ETHUSDTM', 'SOLUSDTM', 'DOTUSDTM', 'ADAUSDTM']
async def run_ticker():
    async def deal_msg(msg):
        # msg = DotMap(msg)
        data = msg['data']
        s = f'TICKER kucoinfutures {data["symbol"]} {data["price"]}'
        # print(s)
        socket.send_string(s)
        
        
    client = WsToken(key='', secret='', passphrase='', is_sandbox=False, url='')
    ws_client = await KucoinFuturesWsClient.create(ticker_loop, client, deal_msg, private=False)
    for symbol in symbols:
        await ws_client.subscribe(f'/contractMarket/ticker:{symbol}')
    # await ws_client.subscribe('/contractMarket/ticker:XBTUSDM')
    # await ws_client.subscribe('/contractMarket/level3:XBTUSDM')


    while True:
        await asyncio.sleep(60)


ticker_loop = asyncio.new_event_loop()

def setup_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

ticker_thread = Thread(target=setup_loop, args=(ticker_loop,))
ticker_thread.start()
ticker_future = asyncio.run_coroutine_threadsafe(run_ticker(), ticker_loop)  

666